# Программа парсер на Python
## Цели
* Скачать access.log : http://www.almhuette-raith.at/apache-log/access.log
* Сформировать CSV файл со списком уникальных IP-адресов из get-запросов. Для каждого из них должно быть указано успешных get-запросов с этого адреса, и количество всех остальных get-запросов.
* Вывести на экран список TOP30 IP адресов, с наибольшим количеством успешных get-запросов.

# Импорт библиотек и загрузка данных

In [1]:
import pandas as pd
import numpy as np
import wget
import re

In [ ]:
wget.download('http://www.almhuette-raith.at/apache-log/access.log')

Откроем access.log и сформируем лист log. После чего удалим из него пустые строки.

In [2]:
with open('access.log') as fp:
    log = []
    [log.append(fp.readline()) for _ in range(10000)]
log.remove('\n')

In [3]:
log[:10]

['13.66.139.0 - - [19/Dec/2020:13:57:26 +0100] "GET /index.php?option=com_phocagallery&view=category&id=1:almhuette-raith&Itemid=53 HTTP/1.1" 200 32653 "-" "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)" "-"\n',
 '157.48.153.185 - - [19/Dec/2020:14:08:06 +0100] "GET /apache-log/access.log HTTP/1.1" 200 233 "-" "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" "-"\n',
 '157.48.153.185 - - [19/Dec/2020:14:08:08 +0100] "GET /favicon.ico HTTP/1.1" 404 217 "http://www.almhuette-raith.at/apache-log/access.log" "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36" "-"\n',
 '216.244.66.230 - - [19/Dec/2020:14:14:26 +0100] "GET /robots.txt HTTP/1.1" 200 304 "-" "Mozilla/5.0 (compatible; DotBot/1.1; http://www.opensiteexplorer.org/dotbot, help@moz.com)" "-"\n',
 '54.36.148.92 - - [19/Dec/2020:14:16:44 +0100] "GET /index.php?option=com_phocagall

* Данные имеют не стандартный отличный от "%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"" формат, поэтому вместо apachelogs.LogParser будем использовать регулярные выражения.

# Формирование CSV файл со списком уникальных IP-адресов из get-запросов

Сформируем датафрейм из трех столбцов: IP-адреса, запросы, из кодов запросов.

Распакуем не пустые списки, присвоив остальным значения NaN чтобы удалить строки их содержащие (останутся только запросы GET).

In [4]:
df = pd.DataFrame(
    {
        'ip': list(map(lambda x: re.findall(r'(?:\d{1,3}[.]){3}(?:\d{1,3})', x), log)),
        'get_post': list(map(lambda x: re.findall(r'GET', x), log)),
        'code': list(map(lambda x: re.findall(r'\s(\d\d\d)\s\d', x), log)),
    }
)

df = df.applymap(lambda x: x[0] if x != [] else np.nan).dropna(axis=0, how='any')
df.sample(5)

,ip,get_post,code
4234,45.138.145.106,GET,200
587,194.156.95.52,GET,200
408,45.145.161.12,GET,200
7536,201.137.179.24,GET,200
38,42.236.10.125,GET,200


In [5]:
# Поменяем тип на int
df.code = df.code.map(int)

Определим столбец success_request, выделив является ли запрос успешным.

In [6]:
df['success_request'] = df.code.apply(lambda x: 200<=x<300)

Сгруппируем данные по ip.

In [7]:
df_pivot = df.pivot_table(index='ip', columns='success_request', values='get_post', aggfunc='count').reset_index()
# Дадим столбцам адекватные имена.
df_pivot.columns = ['ip','other_req','success_req']

# TOP30 IP адресов, с наибольшим количеством успешных get-запросов

In [8]:
df_pivot.sort_values('success_req', ascending=False).head(30)

,ip,other_req,success_req
163,45.144.0.179,NaN,255.0
75,176.222.58.254,NaN,250.0
160,45.138.145.131,NaN,247.0
158,45.132.51.62,NaN,243.0
167,45.153.227.31,NaN,243.0
76,176.222.58.90,NaN,239.0
157,45.132.51.36,NaN,235.0
155,45.132.207.154,NaN,232.0
161,45.138.4.22,NaN,231.0
168,45.153.227.55,NaN,228.0
